In [1]:
!pip install ortools -q
# download codes
!git clone https://github.com/rilianx/eda.git

^C


fatal: destination path 'eda' already exists and is not an empty directory.


## Creación del Modelo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomModel(nn.Module):
    def __init__(self, input_dim, num_heads, head_dim, dropout_rate=0.2):
        super(CustomModel, self).__init__()
        #self.seq_length = seq_length  # Asumiendo una longitud fija de secuencia para simplificar
        self.num_heads = num_heads
        self.head_dim = head_dim

        # Proyección de entrada
        self.input_projection = nn.Linear(input_dim, num_heads * head_dim)

        # Capa de atención multi-cabeza
        self.multihead_attention = nn.MultiheadAttention(embed_dim=num_heads * head_dim,
                                                         num_heads=num_heads,
                                                         dropout=dropout_rate)

        # Capas lineales individuales para cada posición de la secuencia
        # Esto es un cambio respecto al código original para aplicar una capa lineal por posición de salida
        self.positionwise_linear = nn.Linear(num_heads * head_dim, 1)

        # Capa de salida final, después de un flatten, para aplicar Softmax
        # Nota: Softmax se aplica después del flatten, por lo tanto no se define aquí como una capa pero sí en el forward

    def generate_attention_mask(self, x, padding_value=0):
        # Identificar posiciones de padding en x
        mask = (x.sum(dim=-1) == padding_value)  # Asumiendo que el padding se puede identificar sumando los valores de la característica y comparando con 0
        mask = mask.to(dtype=torch.bool)  # Convierte a bool para usar como máscara
        # PyTorch espera una máscara con True y False donde True indica donde aplicar la máscara
        return mask


    def forward(self, x, seq_lengths=10, return_probabilities=False):
        # x: [batch_size, seq_length, input_dim]
        x = x.float()

        max_len = x.shape[1]

        # Generar máscara de atención basada en las longitudes de las secuencias
        attn_mask = None

        # Aplicar proyección de entrada
        x_proj = self.input_projection(x)
        x_proj = x_proj.permute(1, 0, 2)  # Reordenar para multihead_attention: [seq_length, batch_size, num_heads*head_dim]


        # Aplicar atención multi-cabeza
        attn_output, _ = self.multihead_attention(x_proj, x_proj, x_proj, attn_mask=attn_mask)
        attn_output = attn_output.permute(1, 0, 2)  # Reordenar de vuelta: [batch_size, seq_length, num_heads*head_dim]

        # Aplicar la capa lineal posición por posición
        # Usamos una capa lineal que se aplica a cada vector de salida de la atención de forma independiente
        positionwise_output = self.positionwise_linear(attn_output)

        # Flatten
        flat_output = positionwise_output.view(positionwise_output.size(0), -1)  # [batch_size, seq_length]

        # Softmax
        if return_probabilities:
          output = F.softmax(flat_output, dim=-1)
          return output
        else: #return logits
          return flat_output



# Parámetros del modelo
input_dim = 6  # Dimensión de la entrada
num_heads = 10  # Número de cabezas en la atención multi-cabeza
head_dim = 64  # Dimensión de cada cabeza


# Crear el modelo
model = CustomModel(input_dim=input_dim, num_heads=num_heads, head_dim=head_dim)

# Información del modelo (Opcional)
print(model)


## Generación de datos usando **Algoritmo clásico**

In [ ]:
from copy import deepcopy
import random
import math

def distance(punto1, punto2):
    return math.sqrt((punto1[0] - punto2[0])**2 + (punto1[1] - punto2[1])**2)

# función para transformar un estado tsp en una secuencia de vectores
# para el modelo basado en capas de atención
def state2vecSeq(self):
    # creamos dos diccionarios para mantenre un mapeo de los
    # movimientos con los índices de la secuencia del modelo de aprendizaje

    city_locations = self.inst_info.city_locations

    idx2move = dict()
    move2idx = dict()
    origin = city_locations[self.visited[-1]]
    destination = city_locations[self.visited[0]]

    origin_dist = 0.0
    dest_dist = distance(origin, destination)

    seq = [list(origin) + [1,0] + [origin_dist, dest_dist], # Última ciudad visitada (origen)
           list(destination) + [0, 1] + [dest_dist, 0.0]]  # Ciudad final

    idx2move[0] = None
    idx2move[1] = ("constructive", self.visited[0])
    move2idx[self.visited[0]] = 1

    idx = 2
    for i in self.not_visited:
        point = list(city_locations[i])
        origin_dist = distance( point, origin)
        dest_dist = distance( point, destination)
        city_vector = point + [0, 0] + [origin_dist, dest_dist] # Otras ciudades

        seq.append(city_vector)
        idx2move[idx] = ("constructive", i)
        move2idx[i] = idx
        idx += 1

    return seq, idx2move, move2idx

In [ ]:
import numpy as np
from torch.nn.functional import one_hot
from eda.TSP import TSP_Instance, TSP_Environment, TSP_State
from eda.solveTSP_v2 import solve
env = TSP_Environment

def generate_data(max_cities=20, nb_sample=100):
    X = []  # Lista para almacenar las secuencias de entrada
    Y = []  # Lista para almacenar las etiquetas objetivo (las siguientes ciudades a visitar)
    seq_len = max_cities + 1  # Longitud de la secuencia, ajustada para incluir una ciudad extra

    # Bucle para generar datos hasta alcanzar el número deseado de muestras
    while True:
        # 1. Generamos instancia aleatoria
        n_cities = max_cities
        dim = 2  # Dimensión para las coordenadas de la ciudad (2D: x, y)
        city_points = np.random.rand(n_cities, dim)  # Generar puntos aleatorios para las ciudades
        inst_info = TSP_Instance(city_points)

        # 2. Resolvemos TSP usando algoritmo tradicional
        solution = solve(city_points)  # Resolver el TSP y obtener un estado final

        # 3. Iteramos sobre los movimientos de la solución final para generar varias muestras:
        # estado (X) -> movimiento (Y)
        current_state = TSP_State (inst_info)
        env.state_transition(current_state, ("constructive",solution.visited[0]))
        samples_per_sol = max_cities-1  # Número máximo de muestras por solución
        for move in [("constructive", city) for city in solution.visited[1:]]:
            seq, _, move2idx = state2vecSeq(current_state)  # Convertir el estado actual a secuencia vectorizada

            X.append(torch.tensor(seq))  # Añadir la secuencia a X
            Y.append(one_hot(torch.tensor(move2idx[move[1]]), num_classes=seq_len))
            #Y.append(to_categorical(move2idx[move[1]], num_classes=seq_len))  # Añadir el movimiento como categoría a Y

            env.state_transition(current_state, move)  # Hacer la transición al siguiente estado

            # Condiciones de parada basadas en el número de ciudades visitadas/no visitadas o muestras generadas
            if len(current_state.visited) > samples_per_sol or len(X) >= nb_sample:
                break

        # Romper el bucle externo si se ha alcanzado el número deseado de muestras
        if len(X) >= nb_sample:
            break

    X_padded = torch.nn.utils.rnn.pad_sequence(X, batch_first=True)

    return X_padded, torch.stack(Y)

generate_data(5,4)

In [ ]:
from sklearn.model_selection import train_test_split
X,Y=generate_data(max_cities=10, nb_sample=20000)
print(X.shape,Y.shape)

## Entrenamiento del modelo

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd

# Asumiendo que X_padded y Y_stacked ya están definidos y son tensores de PyTorch
dataset = TensorDataset(X, Y)

# Dividir el dataset en entrenamiento y prueba}
train_size = int(0.5 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Definir el modelo, la función de pérdida y el optimizador
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Ciclo de entrenamiento
epochs = 5
# Initialize the DataFrame to store training results
df = pd.DataFrame(columns=["cities", "iter", "Epoch",
                           "Training Loss", "Training Accuracy",
                           "Validation Loss", "Validation Accuracy"])
for num_iter in range(10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0; total = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Limpia los gradientes
            outputs = model(X_batch)  # Obtenemos logits
            loss = loss_function(outputs, y_batch.argmax(dim=1))  # Calcular la pérdida
            loss.backward()  # Backward pass
            optimizer.step()  # Actualizar parámetros
            train_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch.argmax(dim=1)).sum().item()

    
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * correct / total
    
        # Validación
        model.eval()
        validation_loss = 0
        correct = 0; total = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss = loss_function(outputs, y_batch.argmax(dim=1))
                validation_loss += loss.item() * X_batch.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch.argmax(dim=1)).sum().item()
        validation_loss /= len(test_loader.dataset)
        validation_accuracy = 100 * correct / total
        df = pd.concat([df, pd.DataFrame([{
            "cities": 50,
            "iter": num_iter,
            "Epoch": epoch + 1,
            "Training Loss": train_loss,
            "Training Accuracy": train_accuracy,
            "Validation Loss": validation_loss,
            "Validation Accuracy": validation_accuracy
        }])], ignore_index=True)
    
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'Epoch {epoch+1}, Val Loss: {validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%')



## Validación del modelo.
Se utiliza como función de evaluación de movimientos dentro de algoritmo constructivo Greedy

In [ ]:
from eda.TSP import TSP_Instance, TSP_Environment, TSP_State, evalConstructiveActions, plot_tour
from eda.agents import SingleAgentSolver, GreedyAgent
import numpy as np
import pickle

#with open('tsp_model.pkl', 'rb') as archivo:
#    model = pickle.load(archivo)

class ModelEvalActions():
  def __init__(self, model):
    self.model=model

  # permite evaluar acctiones de varios estados a la vez
  # para optimizar los cáluclos del modelo
  def __call__(self, states, env):
    single_state = False
    if not isinstance(states, list):
      single_state=True
      states = [states]

    evals = [list() for _ in states]
    vecSeqs=[]; move2idx =[]

    for state in states:
      vecSeq, _, mov2idx = state.state2vecSeq()
      vecSeqs.append(vecSeq)
      move2idx.append(mov2idx)

    predictions = self.model(torch.tensor(vecSeqs), return_probabilities=True)

    for k in range(len(states)):
      state = states[k]
      for action in env.gen_actions(state, "constructive"):
          idx = move2idx[k][action[1]] #mapping from move to output index (model)
          evals[k].append((action,predictions[k][idx]))

    if single_state: return evals[0]
    else: return evals

# np.random.seed(42)

# creamos un problema con 50 ciudades en un plano 2D
cities  = np.random.rand(500, 2)
inst_info = TSP_Instance(cities)

# referenciamos nuestro ambiente con las "reglas del juego"
env = TSP_Environment
# creamos nuestro agente
greedy = SingleAgentSolver (env,GreedyAgent(ModelEvalActions(model)))
solution, *_ = greedy.solve(TSP_State (inst_info, visited=[0]))
print("Model solution:\n", solution)
plot_tour(cities, solution.visited)


greedy2 = SingleAgentSolver (env,GreedyAgent(evalConstructiveActions))
solution, *_ = greedy2.solve(TSP_State (inst_info, visited=[0]))
print("Greedy solution:\n", solution)
plot_tour(cities, solution.visited)

solution = solve(cities)
print ("OR-Tools:",solution.cost)
plot_tour(cities, solution.visited)


### Múltiples ejecuciones

In [ ]:
from eda.agents import StochasticGreedyAgent

greedy = SingleAgentSolver (env,StochasticGreedyAgent(ModelEvalActions(model), steepness=50))
solutions, *_ = greedy.multistate_solve([deepcopy(TSP_State (inst_info, visited=[0])) for _ in range(10)])

print([s.cost for s in solutions])

best_sol = min(solutions, key=lambda solution: solution.cost)
print("Best solution:\n", best_sol)
plot_tour(cities, best_sol.visited)


In [ ]:
df.to_csv("base_train.csv")

In [ ]:
from tqdm import tqdm
nb_eval = 50
fold = 10
eval_df = pd.DataFrame(columns=["model_name", "cost"])
instances = [
                TSP_Instance(np.random.rand(50, 2)) for _ in tqdm(
                    range(nb_eval), desc="Instances", unit="instance", position=0, leave=True
                )
            ]
greedy = SingleAgentSolver(env, GreedyAgent(ModelEvalActions(model)))
solutions = []

for instance in tqdm(instances, desc="Solving Instances", unit="instance", position=0, leave=True):
    solution, *_ = greedy.solve(TSP_State(instance, visited=[0]))
    eval_df = pd.concat([eval_df, pd.DataFrame([{
        "model_name" : type(model).__name__,
        "iter" : iter_num,        
        "cost": solution.cost,                
}]) ])


In [ ]:
eval_df.reset_index().drop('index', axis=1)